In [2]:
from google.cloud import documentai_v1 as documentai
from vertexai.generative_models import GenerativeModel
import vertexai

from google.cloud import firestore

import os
from dotenv import load_dotenv
import pathlib

import pymupdf
import pymupdf4llm
import pandas as pd
import numpy as np
import random

load_dotenv()
PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID")
LOCATION = os.getenv("GOOGLE_VERTEX_LOC")
PROCESSOR_ID = os.getenv("GOOGLE_DOC_PROCESSOR_ID")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/shash/projects/google_genai/backend/firm-retina-472005-t8-38c1cb563266.json"


name = f"projects/{PROJECT_ID}/locations/{LOCATION}/processors/{PROCESSOR_ID}"


vertexai.init(project=PROJECT_ID,location=LOCATION)


In [3]:
file_path  = "/home/shash/projects/google_genai/data/resumes/computer-science-resume-example.pdf"

In [20]:
document = pymupdf.open(filename=file_path)
text = pymupdf4llm.to_markdown(document)
print(text)

# BELLA TREVINO

### WEB DEVELOPER






### **CAREER OBJECTIVE**

Graduate of computer science with experience working across the
full-stack of software development. Looking for a role where I can
grow and learn from experienced team members while drawing on
project experience I have already successfully executed.

### **WORK EXPERIENCE** Web Developer Intern


Book of the Month
April 2019 - September 2019 / Chicago, IL

## ·

[Developed in-house web app using Node, React, and HTML/CSS]

that allowed employees to manage and recommend books

## ·

[Gathered user requirements, and built and maintained code for]

the tool, saving employees 14 monthly hours in labor

## ·

[Collaborated with 3 interns and senior developer to brainstorm]

and implement ideas and feedback into app's development

## ·

[Selected as the top performing intern by the technology team]

### **PROJECTS** Social Media Scheduler


Creator

## ·

[Built responsive app using Django and Node that allowed users]

to sch

Resume Parsing



In [21]:

model_prompt = f"""
You are given an resume, you have to parse the below text and return in JSON format with correct entities in correct fields.
1.name , 2.education , 3.skills ,4.projects
RESUME TEXT: {text}
ONLY RESPOND IN VALID JSON, DO NOT include any extra abbrivations,salutations,or extra text.
Follow the below format:
```json
{{
  
  "name":
  "education":[{{"degree":, "branch":, "college":}}],
  "skills":[...],
  "projects/experience":[{{"title":"","desc":"..."}}]
}}
```
"""

model = GenerativeModel(model_name="gemini-2.5-flash")

res = model.generate_content(model_prompt)
print(res.text)

```json
{
  "name": "BELLA TREVINO",
  "education": [
    {
      "degree": "Computer Science",
      "branch": "Computer Science",
      "college": "University of Illinois Chicago"
    }
  ],
  "skills": [
    "Node",
    "React",
    "HTML/CSS",
    "Django",
    "scikit-learn",
    "Python",
    "Instagram API",
    "Twitter API"
  ],
  "projects/experience": [
    {
      "title": "Web Developer Intern at Book of the Month",
      "desc": "Developed in-house web app using Node, React, and HTML/CSS that allowed employees to manage and recommend books. Gathered user requirements, and built and maintained code for the tool, saving employees 14 monthly hours in labor. Collaborated with 3 interns and senior developer to brainstorm and implement ideas and feedback into app's development. Selected as the top performing intern by the technology team."
    },
    {
      "title": "Social Media Scheduler",
      "desc": "Built responsive app using Django and Node that allowed users to schedu

16 FPR Algorithm and workflow

In [5]:
mbti_questions_path = "/home/shash/projects/google_genai/data/MBTI_questions/MBTI.csv"

In [6]:
df = pd.read_csv(mbti_questions_path)
df.drop(df.columns[1],axis=1)
df.head()

,Unnamed: 0,Type,Sentence,labels
0,0,INFJ,I tend to empathize deeply with the emotions o...,8
1,1,INFJ,I often daydream and explore complex ideas in ...,8
2,2,INFJ,I'm highly sensitive to criticism and take it ...,8
3,3,INFJ,I believe in the power of intuition and gut fe...,8
4,4,INFJ,I can be reserved initially but open up in mea...,8


{'INFJ': ["I'm interested in psychology and understanding human behavior."], 'INTP': ['I enjoy experimenting and exploring new ways of doing things.'], 'INTJ': ['I enjoy exploring complex theories and understanding underlying principles.'], 'ENTP': ['I enjoy exploring different perspectives and viewpoints.'], 'ISTP': ["I'm curious about how things are built or constructed."], 'INFP': ["I'm compassionate and often put others' needs before my own."], 'ENTJ': ["I'm strategic and always thinking about the next steps."], 'ESTP': ["I'm confident in my abilities and trust my instincts."], 'ENFJ': ["I'm good at understanding people's motivations and needs."], 'ENFP': ["I'm optimistic and see the good in people and situations."], 'ESTJ': ["I'm responsible and take my duties seriously."], 'ESFJ': ["I'm organized and enjoy planning social events and gatherings."], 'ESFP': ["I'm friendly and approachable, making it easy for others to connect with me."], 'ISTJ': ["I'm reserved in expressing emotion

In [ ]:


db = firestore.Client(database="hackathonfirestore")
batch = db.batch()
collection_ref = db.collection("questions")

for index,row in df.iterrows():
    ques = row["Sentence"]
    cat = row["Type"]
    dat = {"question": ques,"type":cat}
    # if index>10:
    #     break
    try:
        # db.collection("questions").add(dat)
        doc_ref = collection_ref.document()
        batch.set(doc_ref,{
            "question":row["Sentence"],
            "type":row["Type"]
        })
        print(f"inserted records {index}",end='\r')

    except Exception as e:
        print(f"error in storing data in firestore {str(e)}")
        print(f"type: {type(e)}")
        continue

batch.commit()
print("Batch insert done")


Batch insert done460


In [12]:
import google.auth
creds, project = google.auth.default()
print("Active project:", project)


Active project: firm-retina-472005-t8


In [16]:
docs= db.collection("questions").stream()
data = []
for doc in docs:
    data.append(doc.to_dict())

ques_df = pd.DataFrame(data)

ques_list = {}
for category in ques_df['type'].unique():
    questions = ques_df.loc[ques_df['type']==category,'question'].tolist()
    ques = random.choice(questions)
    ques_list[category] = [ques]

print(ques_list)


{'INTJ': ['I enjoy strategic planning and thinking about the future.'], 'ENTP': ['I enjoy debating and challenging ideas in a lively discussion.'], 'ENFJ': ['I enjoy creating a nurturing and supportive environment for others.'], 'ESTJ': ["I'm focused on efficiency and streamlining processes for better results."], 'INFP': ["I'm sensitive to criticism and take it to heart."], 'ESFJ': ["I'm loyal to my values and seek to maintain harmony in relationships."], 'ESTP': ["I'm resourceful and can find creative solutions to challenges."], 'ISTJ': ["I'm loyal to my values and principles, and I stand by them firmly."], 'ENTJ': ["I'm decisive and can make tough decisions even under pressure."], 'ISTP': ["I'm interested in how things are constructed and function."], 'ISFP': ['I enjoy exploring new experiences and living in the moment.'], 'INFJ': ['I have a vivid imagination and enjoy exploring creative projects.'], 'INTP': ['I can get lost in researching a topic of interest for hours.'], 'ISFJ': ["

In [ ]:
#weighing algo

#input data : {TYPE,SCORE}

# strongly agree: 5 , strongly disagree: 1

dummy_answers = [
    {"INFJ": 5},  # Q1
    {"ENFP": 3},  # Q2
    {"INFP": 4},  # Q3
    {"ENTP": 2},  # Q4
    {"ISFJ": 5},  # Q5
    {"ISTJ": 4},  # Q6
    {"INTJ": 3},  # Q7
    {"ESFP": 2},  # Q8
    {"ESTP": 1},  # Q9
    {"ENTJ": 4},  # Q10
    {"INFJ": 3},  # Q11
    {"ENFP": 5},  # Q12
    {"INFP": 2},  # Q13
    {"ENTP": 4},  # Q14
    {"ISFJ": 3},  # Q15
    {"ISTJ": 2},  # Q16
    {"INTJ": 5},  # Q17
    {"ESFP": 3},  # Q18
    {"ESTP": 4},  # Q19
    {"ENTJ": 5}   # Q20
]

personality = {}
for entry in dummy_answers:
    for key,value in entry.items():
        if key not in personality.keys():
            personality[key] = value
        else:
            personality[key] += value

user_pers = []
max_val = 0
for key,value in personality.items():
    if value>= max_val:
        max_val = value
for key,value in personality.items():
    if value==max_val:
        user_pers.append(key)

print(personality)
print(user_pers)

{'INFJ': 8, 'ENFP': 8, 'INFP': 6, 'ENTP': 6, 'ISFJ': 8, 'ISTJ': 6, 'INTJ': 8, 'ESFP': 5, 'ESTP': 5, 'ENTJ': 9}
['ENTJ']
